In [1]:
import pandas as pd
from Func_extr import *
from Func_viz import *
from Func_analisis  import *     
import sqlite3

In [13]:
conn = sqlite3.connect('BBDD/aguaCHJucar.db')

cursor = conn.cursor()

query = f'''
        SELECT location_id, latitude, longitude FROM locations_id WHERE Type = "Copernicus";
    '''

df_loc = pd.read_sql_query(query, conn)
# Ejecutar la consulta
cursor.execute(query)
conn.close()


In [60]:
pixels_corrPos  = []
pixels_corrNeg  = []
for pixel in df_loc['location_id']:
    df = create_df(pixel)
    if df.empty:
        pass
    else:
        variables = ['total_precipitation']
        df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
        df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
        #df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
        df = df.sort_values('date').set_index('date')
        df = df.select_dtypes(exclude=['datetime64'])
        correlaciones = df.corr()
        prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]
        media_corr = correlaciones[prec_columns].T['quantity_hm3'].mean()
        if media_corr>0:
            pixels_corrPos.append(pixel)
        else:
            pixels_corrNeg.append(pixel)
df_corr_neg = df_loc[df_loc['location_id'].isin(pixels_corrNeg)]

In [84]:
import folium

m = folium.Map(location=[39.1, -1.2], zoom_start=8)

for index,row in df_corr_neg.iterrows():
    folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"location_id: {row['location_id']}",
            icon=folium.Icon(color='red')
        ).add_to(m)
m    

In [50]:
df = create_df(189)
if df.empty:
    pass
else:
    variables = ['total_precipitation']
    df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
    df = df.sort_values('date').set_index('date')
    df = df.select_dtypes(exclude=['datetime64'])
    correlaciones = df.corr()
    prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]
    media_corr = correlaciones[prec_columns].T['quantity_hm3'].mean()

In [56]:
df = create_df(185)
if df.empty:
    pass
else:
    variables = ['total_precipitation']
    df = retardAvg_tNat(df, vars = variables, lags = [1,7,15,20], frec = 'D')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3,4], frec = 'M')
    df = retardAvg_tNat(df, vars = variables, lags = [1,2,3], frec = 'Y')
    df = df.sort_values('date').set_index('date')
    df = df.select_dtypes(exclude=['datetime64'])
    correlaciones = df.corr()
    prec_columns = [i for i in correlaciones.columns if 'precipitation' in i]